##Final Project : Discrete Optimisation- CS 2019
## English Peg Solitaire
### Iheb Belgacem

In this project we want to find solutions for solvable peg solitaire games. 
The english version contains 33 cells. The goal is to find if there is a series of legal jumps that allows us to get from an initial configuration to a final configuration. </br>
</br>
                          ![Texte alternatif…](http://www.home.hs-karlsruhe.de/~pach0003/informatik_1/aufgaben/solitaer.jpg)
                         
As explained in the scientific article written by Masashi KIYOMI, Tomomi MATSUI from the university of Tokyo, the peg solitaire can be modeled by the following Mixed Integer Program : 
</br>
$$
\begin{split}
    IP1: & \text{find} \quad & (x^1, ... , x^l) &\\
               & s.t.      &A(x^1+ ... + x^l) = p_s - p_f ,&\\
               &            &0 \le p_s - (x^1+ ... + x^k) \le 1 , \quad &\forall k \in \{1, ..., l\} \\
               &            &x^k_1 + ... + x^k_m = 1 , \quad           &\forall k \in \{1, ..., l\} \\
               &            &x^k \in \{0,1\}^m   & \forall k \\
\end{split}
$$
</br>
However solving this problem can take a lot of time, so we chose to implement the second method proposed by Kiomi and Matsui, which is based on a back-tracking search Algorithm. (see the solvprog function) </br>
In order to prune the search space we used two different methods: 
1.    First of all, it's proven that there is an upper bound for how many times each jump j is used in the path to get from the intial to the final configuration. 
This upper band is the solution for the following MIP: </br>
   $$
   \begin{split}
       RUBj: & \max. \quad &x_j \\
                  & s.t.                &Ax = p_s - p_f    \\
                  && x = (x_1+ ...+ x_m) \text{ non negative integers}
   \end{split}
   $$
We have used the solutions for this MIP (saved in an array) to prune the search space. 
If RUBj is unfeasible, the peg solitaire problem with the given intial and final configuration doesn't have any solution
2.  We have also used  a hash table where we have stored every configuartion of the cells that have shown to lead to no solution in order to speed up the backtracking algorithm .(see sansissue and solvprog functions).

</br>
</br>
We have chosen two ways to represent every configuration of the cells : the first one using a pandas dataframe (which is more pleasent visually) and the second one is an array. The cells are represented in this array in a logical order starting from the upper left of the game. Empty cells are represented by 0, the other ones are represented by the value one. </br>
We have used to solve the MIP RUBj the OR-tools developped by Google. (see the solving function)

In [0]:
import numpy as np
import pandas as pd


In [0]:
#These functions help us to go from the coordinates of a cell in (x,y) to the corresponding coordinates in a 1D vector and vice versa
def unravel (i):
  if i<3:
    (x,y)=(0,2+i)
  elif i<6:
    (x,y)=(1,i-1)
  elif i<27:
    (x,y)=(((i+1)//7)+1,((i+1)%7))
  elif i<30:
    (x,y)=(5,i-25)
  else :
    (x,y)=(6,i-28)
  return (x,y)

def ravel(a,b):
  z=0
  if a==0:
    z=b-2
  elif a==1:
    z=b+1
  elif a<5:
    z=7*(a-1)+b-1
  elif a==5:
    z=25+b
  else :
    z=28+b
  return z

In [0]:
#nb of cells
nc=33
p=np.zeros(nc)

In [0]:
#This is an example for an intial and a final configuration.
ps1=np.ones(33)
ps1[16]=0
pf1=np.ones(33)
pf1[ravel(3,4)]=0
pf1[ravel(4,5)]=0
pf1[ravel(5,3)]=0
pf1[ravel(5,4)]=0
pf1[ravel(7,2)]=0
pf1[ravel(7,3)]=0
pf1[ravel(7,4)]=0




In [0]:
# an empty cell is represented by 0, and a non-empty one with 1
def afficher(ps):
  sh=np.ones((7,7))
  sh=2*sh
  for i in range(len(ps)):
    sh[unravel(i)[0]][unravel(i)[1]]=ps[i]
    df=pd.DataFrame(sh)
  df=df.replace([2],[""])
  print(df)

In [0]:
afficher(ps1)
afficher(pf1)

   0  1  2  3  4  5  6
0        1  1  1      
1        1  1  1      
2  1  1  1  1  1  1  1
3  1  1  1  0  1  1  1
4  1  1  1  1  1  1  1
5        1  1  1      
6        1  1  1      
   0  1  2  3  4  5  6
0        1  1  1      
1        1  1  1      
2  1  1  1  1  1  1  1
3  1  1  1  1  0  1  1
4  1  1  1  1  1  0  1
5        1  0  0      
6        0  0  0      


In [0]:
#Function that tells us if (a,b) are valid coordinates for our problem
def yon(a,b):
  test=True
  if a<2 or a>4 :
    if (b<2) or (b>4):
      test=False
  if a>6:
    test=False
  if a<0:
    test=False
  if b<0:
    test=False
  if b>6:
    test=False
  return test

In [0]:
#Function that tells us if a certain type of moves is to be considered or not in our probem ( from example west(0,0) can never be a valid move)
def down(a,b):
  test=False
  test= (yon (a,b) and yon(a+1,b) and yon(a+2,b))
  return test

def up(a,b):  
  test=False
  test= (yon (a,b) and yon(a-1,b) and yon(a-2,b))
  return test

def east(a,b):
  test=False
  test= (yon (a,b) and yon(a,b+1) and yon(a,b+2))
  return test

def west(a,b):
  test=False
  test= (yon (a,b) and yon(a,b-1) and yon(a,b-2))
  return test

In [0]:
dir=[1,2,3,4] #direction 1 is down, 2 is up, 3 is est , 4 is west
r=np.linspace(0,32,33,dtype=int)
#j contains all the different jumps
j=[]
for i in r:
  if up(unravel(i)[0],unravel(i)[1]) :
    j.append([i,2])
  if down(unravel(i)[0],unravel(i)[1]) :
    j.append([i,1])
  if east(unravel(i)[0],unravel(i)[1]) :
    j.append([i,3])
  if west(unravel(i)[0],unravel(i)[1]) :
    j.append([i,4])

In [0]:
#Let's build the A matrix
A=np.zeros((nc,len(j)))
for t in range(len(j)):
  if j[t][1]==1:#down
    A[j[t][0]][t]=1
    xy=unravel(j[t][0])
    A[ravel(xy[0]+1,xy[1])][t]=1
    A[ravel(xy[0]+2,xy[1])][t]=-1
  if j[t][1]==2:#up
    A[j[t][0]][t]=1
    xy=unravel(j[t][0])
    A[ravel(xy[0]-1,xy[1])][t]=1
    A[ravel(xy[0]-2,xy[1])][t]=-1
  if j[t][1]==3:#east
    A[j[t][0]][t]=1
    xy=unravel(j[t][0])
    A[ravel(xy[0],xy[1]+1)][t]=1
    A[ravel(xy[0],xy[1]+2)][t]=-1
  if j[t][1]==4:#west
    A[j[t][0]][t]=1
    xy=unravel(j[t][0])
    A[ravel(xy[0],xy[1]-1)][t]=1
    A[ravel(xy[0],xy[1]-2)][t]=-1
A.shape    

(33, 76)

In [0]:
!pip3 install ortools

     |████████████████████████████████| 27.1MB 42.7MB/s 


In [0]:
from __future__ import print_function
from ortools.linear_solver import pywraplp

In [0]:
#With solving (and the following for loop ) we are going to build a table with the solutions of MIP that will help us prune the search space in the backtracking algorithm we are going to use later
def solving(A,ps,pf,ind):
  solver = pywraplp.Solver('upperbound',pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
  infinity = solver.infinity()
  jp={}
  for i in range(A.shape[1]):
    jp[i]=solver.IntVar(0.0, infinity,'jp[%i]' % i)
  for i in range(A.shape[0]):
    solver.Add(solver.Sum([A[i][j]*jp[j] for j in range(A.shape[1])]) == ps[i]-pf[i])
  solver.Maximize(jp[ind])
  result_status = solver.Solve()
  return solver.Objective().Value()




In [0]:
upper=[]
for i in range(A.shape[1]):
  upper.append(solving(A,ps1,pf1,i))

In [0]:
print("We are going to use the following table representing the upper bound for the number of times each jump is used to prune the search space from the backtracking algorithm")
print(upper)

We are going to use the following table representing the upper bound for the number of times each jump is used to prune the search space from the backtracking algorithm
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0]


In [0]:
#resulmove helps us get the state of the game at n+1 given the state of the game at n and the jump to be done, reversemove does the opposite job
def resulmov(A,pos,j,ind):
  vect=np.zeros(len(j))
  vect[ind]=1
  pp=pos-np.dot(A,vect)
  return pp
def reversemove(A,pos,j,ind):
  vect=np.zeros(len(j))
  vect[ind]=1
  pp=pos+np.dot(A,vect)
  return pp


In [0]:
#possible is a function that tells us if a jump is possible or not given the state of the game at n
def possible(pos,jump):
  p=pos
  test=False
  if jump[1]==1: #down
    t1= (p[jump[0]]==1)
    (a,b)=unravel(jump[0])
    t2= (p[ravel(a+1,b)]==1)
    t3= (p[ravel(a+2,b)]==0)
    test=( t1 and t2 ) and t3
  elif jump[1]==2 :#up
    t1= (p[jump[0]]==1)
    (a,b)=unravel(jump[0])
    t2= (p[ravel(a-1,b)]==1)
    t3= (p[ravel(a-2,b)]==0)
    test=( t1 and t2 ) and t3
  elif jump[1]==3 :#east
    t1= (p[jump[0]]==1)
    (a,b)=unravel(jump[0])
    t2= (p[ravel(a,b+1)]==1)
    t3= (p[ravel(a,b+2)]==0)
    test=( t1 and t2 ) and t3
  else:#west
    t1= (p[jump[0]]==1)
    (a,b)=unravel(jump[0])
    t2= (p[ravel(a,b-1)]==1)
    t3= (p[ravel(a,b-2)]==0)
    test=( t1 and t2 ) and t3
  return test

In [0]:
#This function helps us create the hash table we will use to speed up the back-tracking algorithm
def sansissue(A,j,ind,ps,hashtable):
  test=True
  psp=resulmov(A,ps,j,ind)
  if len(hashtable)>0:
    for i in range(len(hashtable)):
      if np.array_equal(psp,hashtable[i]):
        test=False
  return test
  

In [0]:
#Backtracking algorithom to solve the game
def solvprog(ps,pf,j,A,path,upper,nbusejump,hashtable):
  if np.sum(ps)==np.sum(pf) :
    afficher(pf)
    if np.array_equal(ps,pf)==False:
      hashtable.append(ps)
    if np.array_equal(ps,pf)==True:
      print("The problem is solvable and the path is")
      print(path)
      print("The length of the path is :")
      print(len(path))
    return np.array_equal(ps,pf)
  else:
    for i in range(len(j)):
      test=sansissue(A,j,i,ps,hashtable)
      if possible(ps,j[i]) and (nbusejump[i]<=upper[i]) and (upper[i]>0) and test:
        print("path from",unravel(j[i][0]), "direction ", j[i][1])
        pp=resulmov(A,ps,j,i)
        afficher(pp)
        ps=pp
        path.append(j[i])
        print(path)
        nbusejump[i]=nbusejump[i]+1
        found=solvprog(ps,pf,j,A,path,upper,nbusejump,hashtable)
        if found :
          return True
          break
        else:
          print("go back")
          ps=reversemove(A,ps,j,i)
          nbusejump[i]=nbusejump[i]-1
          afficher(ps)
          path.pop()
    return False   
        

In [0]:
afficher(ps1)
afficher(pf1)

   0  1  2  3  4  5  6
0        1  1  1      
1        1  1  1      
2  1  1  1  1  1  1  1
3  1  1  1  0  1  1  1
4  1  1  1  1  1  1  1
5        1  1  1      
6        1  1  1      
   0  1  2  3  4  5  6
0        1  1  1      
1        1  1  1      
2  1  1  1  1  1  1  1
3  1  1  1  1  0  1  1
4  1  1  1  1  1  0  1
5        1  0  0      
6        0  0  0      


In [0]:

nbusejump=np.zeros(A.shape[1])
#direction 1 is down, 2 is up, 3 is est , 4 is west
solvprog(ps1,pf1,j,A,[],upper,nbusejump,[])

path from (5, 3) direction  2
   0  1  2  3  4  5  6
0        1  1  1      
1        1  1  1      
2  1  1  1  1  1  1  1
3  1  1  1  1  1  1  1
4  1  1  1  0  1  1  1
5        1  0  1      
6        1  1  1      
[[28, 2]]
path from (4, 5) direction  4
   0  1  2  3  4  5  6
0        1  1  1      
1        1  1  1      
2  1  1  1  1  1  1  1
3  1  1  1  1  1  1  1
4  1  1  1  1  0  0  1
5        1  0  1      
6        1  1  1      
[[28, 2], [25, 4]]
path from (6, 4) direction  2
   0  1  2  3  4  5  6
0        1  1  1      
1        1  1  1      
2  1  1  1  1  1  1  1
3  1  1  1  1  1  1  1
4  1  1  1  1  1  0  1
5        1  0  0      
6        1  1  0      
[[28, 2], [25, 4], [32, 2]]
path from (3, 4) direction  1
   0  1  2  3  4  5  6
0        1  1  1      
1        1  1  1      
2  1  1  1  1  1  1  1
3  1  1  1  1  0  1  1
4  1  1  1  1  0  0  1
5        1  0  1      
6        1  1  0      
[[28, 2], [25, 4], [32, 2], [17, 1]]
path from (6, 2) direction  3
   0  1  2  3  4  5 

True